All the imports

In [ ]:
import pandas as pd
import numpy as np
import polars as pl
from matplotlib import pyplot as plt
from src.utils.analysis_tools import *


In [2]:
path = "./data/"
filtered_meta = pl.read_csv(path+"final_yt_metadata.csv",infer_schema=False)

In [ ]:
plot_video_stat(filtered_meta, 'duration')
plot_video_stat(filtered_meta, 'view_count')
plot_video_stat(filtered_meta, 'like_count')
plot_video_stat(filtered_meta, 'dislike_count')

In [ ]:
plot_most_common_words(filtered_meta, 'title', 50)


In [ ]:
plot_most_common_words(filtered_meta, 'description', 50)

In [ ]:
plot_text_len_char(filtered_meta, 'title')
plot_text_len_char(filtered_meta, 'description')

In [ ]:
plot_most_common_tags(filtered_meta, 50)


In [ ]:
high_likes= highperformer(filtered_meta, 'view_count', 1)
plot_video_stat(high_likes, 'duration')

In [ ]:
plot_text_len_words(filtered_meta, 'description')
plot_text_len_words(filtered_meta, 'title')